# Projet maison n° 3

In [9]:
# imports
import pandas as pd
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from progress_bar import start_loading,update_Loader,finish_loader,sleep

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).


In [10]:
# names us
def df_names_us():
    if os.path.isfile('us_name_df_orig.pickle'):
        return pd.read_pickle('us_name_df_orig.pickle')

    dir_list = os.listdir('names/')
    frames =[]
    start_loading(f"Load files from names/")
    for idx,txt_dir in enumerate(dir_list):

        if not txt_dir.endswith('.txt'):
            #print(f'WARNING file {txt_dir} nearly caused havock!!!!!')
            #print('You bloody lucky i caught it, thank me later')
            continue

        temp_df = pd.read_csv(f'names/{txt_dir}',names = ['name', 'gender', 'births'])
        temp_df.insert(0,'Year',txt_dir.replace('.txt','').replace('yob',''))

        frames.append(temp_df)

        #print(f'{idx}/{ len(dir_list)} input :{txt_dir}')

        us_name_df = pd.concat(frames, axis=0)
        us_name_df.to_pickle('us_name_df_orig.pickle')
        update_Loader(f"Load files from names/",int(idx/3),int(len(dir_list)/3))
        #print(f'{idx}/{ len(dir_list)} input :{txt_dir}')
    finish_loader(f"Load files from names/")
    return us_name_df

In [11]:
us_baby_names_df = df_names_us()


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [12]:
# names fr
def df_names_fr():
    french_name_dir = pd.read_csv("nat2020.csv",sep=";", header=None,skiprows = 1,names=['gender','name','year','births'],dtype={'gender':'str','name':'str','births':'int'})
    mapping_gender = {1:"M",2:"F"}
    french_name_dir["gender"]=french_name_dir["gender"].map(mapping_gender)
    french_name_dir.loc[french_name_dir['name'].isna().values ,'name'] = 'NA'
    french_name_dir=french_name_dir[(french_name_dir['name']!="_PRENOMS_RARES") & (df['name'].str.len()>1)]
    french_name_dir=french_name_dir[french_name_dir['year']!="XXXX"]
    french_name_dir['year']=french_name_dir['year'].astype(int)
    french_name_dir=french_name_dir[['year','name','gender','births']]
    french_name_dir['name']=french_name_dir['name'].str.capitalize()
    french_name_dir=french_name_dir.sort_values(['year','gender','births','name'],ascending=(True,True,False,True)).reset_index(drop=True)
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [13]:
# taux de change
def df_taux_change(devises):
    exchange_df = pd.read_csv('Webstat_Export_20210929.csv',sep=";",skiprows=[1,2,3,4,5],index_col='Titre :',na_values=['-',''])
    exchange_df.index=pd.to_datetime(exchange_df.index,format='%d/%m/%Y')
    df=exchange_df.rename_axis('Date')
    df=df.replace(',','.',regex=True).astype(float)
    df = pd.DataFrame(df[devises],columns=devises)
    return df

### Tests

In [14]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [15]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [16]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ERROR
test_df_names_us (__main__.Lesson4Tests) ... FAIL
test_df_taux_change (__main__.Lesson4Tests) ... ERROR

ERROR: test_df_names_fr (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/9f/p0t58rcj5m19q_p7696wb52w0000gp/T/ipykernel_5571/2291148488.py", line 16, in test_df_names_fr
    df = df_names_fr()
  File "/var/folders/9f/p0t58rcj5m19q_p7696wb52w0000gp/T/ipykernel_5571/2767033264.py", line 7, in df_names_fr
    french_name_dir=french_name_dir[(french_name_dir['name']!="_PRENOMS_RARES") & (df['name'].str.len()>1)]
NameError: name 'df' is not defined

ERROR: test_df_taux_change (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/9f/p0t58rcj5m19q_p7696wb52w0000gp/T/ipykernel_5571/2291148488.py", line 31, in test_df_taux_change
    df = df_taux_